In [1]:
import pandas as pd
import time
from pulp import *

inputFileName = "data/data_transportationPb_mini.xlsx"
paramDF = pd.read_excel(inputFileName, "Param", skiprows=0)
Trips = pd.read_excel(inputFileName, "Trips", skiprows=0)
Distance = pd.read_excel(inputFileName, "Distance", skiprows=0)

Trips.dropna(how='all', inplace=True)
Distance.dropna(how='all', inplace=True)

NbHome = paramDF["Value1"][4]
NbWork = paramDF["Value1"][5]
Time_per_station = paramDF["Value1"][6]
Speed = paramDF["Value1"][7]
List_Vehicule = [paramDF.iloc[0]["Value1"],
                 paramDF.iloc[0]["Value2"],
                 paramDF.iloc[0]["Value3"]]
List_Nb_Vehicules = [paramDF.iloc[1]["Value1"],
                 paramDF.iloc[1]["Value2"],
                 paramDF.iloc[1]["Value3"]]
List_Capacity = [paramDF.iloc[3]["Value1"],
                 paramDF.iloc[3]["Value2"],
                 paramDF.iloc[3]["Value3"]]
List_Cost = [paramDF.iloc[2]["Value1"],
                 paramDF.iloc[2]["Value2"],
                 paramDF.iloc[2]["Value3"]]

List_of_Homes = list(Trips["Departure"].dropna().unique())
List_of_Sites = list(Trips["Arrival"].dropna().unique())
List_Of_Stations = List_of_Homes + List_of_Sites
List_of_jouneys = []
List_of_jouney_Cost = []
List_of_journeys_Capacities = []
for indice, type_vehicule in enumerate(List_Vehicule):
    for number in range(List_Nb_Vehicules[indice]):
        List_of_jouneys.append(type_vehicule + "#" + str(number))
        List_of_jouney_Cost.append(List_Cost[indice])
        List_of_journeys_Capacities.append(List_Capacity[indice])
        
Trips_Dict = dict()
Distance_Dict = dict()
for i in range(Trips.shape[0]):
    key = Trips.iloc[i]['Departure'] + "#" + Trips.iloc[i]['Arrival']
    Trips_Dict[key] = Trips.iloc[i]['NbEmployees']

for i in range(Distance.shape[0]):
    key = Distance.iloc[i]['Departure'] + "#" + Distance.iloc[i]['Arrival']
    Distance_Dict[key] = Distance.iloc[i]['Distance']
    key_inv = Distance.iloc[i]['Arrival'] + "#" + Distance.iloc[i]['Departure']
    Distance_Dict[key_inv] = Distance.iloc[i]['Distance']

In [2]:
# CREATE LINEAR PROGRAM

prob = LpProblem("VRP Problem", LpMinimize)

# ADD DEcision VARIABLES

Distance = LpVariable.dicts("Distance", (journey for journey in List_of_jouneys ), cat='Continuous')

Allocation = LpVariable.dicts("Allocation", ((journey, station) for journey in List_of_jouneys for station in List_Of_Stations), cat='Binary')

Same_Journey = LpVariable.dicts("Same_Journey", ((journey, home, site) for journey in List_of_jouneys for home in List_Of_Stations for site in List_Of_Stations), cat='Binary')

IsFirst = LpVariable.dicts("IsFirst", ((journey, station) for journey in List_of_jouneys for station in List_Of_Stations), cat='Binary')

IsSuccessorVar = LpVariable.dicts("IsSuccessor", ((journey, station1, station2) for journey in List_of_jouneys for station1 in List_Of_Stations for station2 in List_Of_Stations), cat='Binary')


# ADD OBJECTIVE Function
Max_Durantion = LpVariable("Max_Duration", cat="Continuous")
prob += Max_Durantion

# ADD CONSTRAINTS
# 1- Define Same_Journey
for journey in List_of_jouneys:
    for home in List_of_Homes:
        for site in List_of_Sites:
            prob += Same_Journey[journey, home, site] <= Allocation[journey, site]
            prob += Same_Journey[journey, home, site] <= Allocation[journey, home]



#At leat one journey linking home to work
for home in List_of_Homes:
    for site in List_of_Sites:
        if home + "#" + site in Trips_Dict.keys():
            if Trips_Dict[home + "#" + site] > 0:
                prob += lpSum(Same_Journey[journey, home, site] for journey in List_of_jouneys) >= 1

#Define Distance
for journey in List_of_jouneys:
    prob += Distance[journey] == lpSum(IsSuccessorVar[journey, station1, station2]*Distance_Dict[station1 + "#" + station2] for station1 in List_Of_Stations for station2 in List_Of_Stations if station1 != station2)

# Define Max durantion
for journey in List_of_jouneys:
    prob += Speed*Distance[journey] + Time_per_station*lpSum(Allocation[journey, home] for home in List_of_Homes) <= Max_Durantion

#Either first or successor A voir generaliser pour toutes les stations
for journey in List_of_jouneys:
    for station1 in List_Of_Stations:
        prob += IsFirst[journey, station1] + lpSum(IsSuccessorVar[journey, station2, station1] for station2 in List_Of_Stations if station1 != station2) >= Allocation[journey, station1]
        #Successor or predec only if its allocated to the journey
        prob += lpSum(IsSuccessorVar[journey, station2, station1] for station2 in List_Of_Stations if station1 != station2) <= Allocation[journey, station1]
        prob += lpSum(IsSuccessorVar[journey, station1, station2] for station2 in List_Of_Stations if station1 != station2) <= Allocation[journey, station1]

        if station1 in List_of_Sites:
            prob += IsFirst[journey, station1] == 0

#At most one is First per journey
for journey in List_of_jouneys:
    prob += lpSum(IsFirst[journey, station] for station in List_Of_Stations) <= 1

#At most one successor
for journey in List_of_jouneys:
    for station1 in List_Of_Stations:
        prob += lpSum(IsSuccessorVar[journey, station1, station2] for station2 in List_Of_Stations) <= 1

C:\Users\taoufik.elkhiraoui\Anaconda3\envs\xeus-python\lib\site-packages\pulp\pulp.py:1199: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [3]:
prob.solve()
if LpStatus[prob.status] == "Optimal":  # 1 for Optimal, 0 for "Not Solved", -1 for "Infeasible", -2 "Unbounded", -3 "Undefined"
    print("Objective = ", value(prob.objective))
    varsDict = {}
    for v in prob.variables():
        varsDict[v.name] = v.varValue
        if v.varValue != 0:
            print(v.name, "=", v.varValue)


Objective =  52.6
Allocation_('Small#0',_'H1') = 1.0
Allocation_('Small#0',_'H2') = 1.0
Allocation_('Small#0',_'H4') = 1.0
Allocation_('Small#0',_'W1') = 1.0
Allocation_('Small#0',_'W3') = 1.0
Allocation_('Small#1',_'H1') = 1.0
Allocation_('Small#1',_'H2') = 1.0
Allocation_('Small#1',_'H3') = 1.0
Allocation_('Small#1',_'W1') = 1.0
Allocation_('Small#1',_'W2') = 1.0
Distance_Small#0 = 18.8
Distance_Small#1 = 18.6
IsFirst_('Small#0',_'H4') = 1.0
IsFirst_('Small#1',_'H1') = 1.0
IsSuccessor_('Small#0',_'H1',_'H2') = 1.0
IsSuccessor_('Small#0',_'H2',_'W1') = 1.0
IsSuccessor_('Small#0',_'H4',_'H1') = 1.0
IsSuccessor_('Small#0',_'W1',_'W3') = 1.0
IsSuccessor_('Small#1',_'H1',_'H2') = 1.0
IsSuccessor_('Small#1',_'H2',_'W1') = 1.0
IsSuccessor_('Small#1',_'H3',_'W2') = 1.0
IsSuccessor_('Small#1',_'W2',_'H3') = 1.0
Max_Duration = 52.6
Same_Journey_('Small#0',_'H1',_'W1') = 1.0
Same_Journey_('Small#0',_'H1',_'W3') = 1.0
Same_Journey_('Small#0',_'H2',_'W1') = 1.0
Same_Journey_('Small#0',_'H2',_'W3'